In [1]:
# use classification to predict lybrary 
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from collections import OrderedDict
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [2]:
# data ingetion

data = pd.read_csv(r'C:\Heart_Attack_Risk_Model\data\raw\cardiovascular_risk_dataset.csv')
data


,Patient_ID,age,bmi,systolic_bp,diastolic_bp,cholesterol_mg_dl,resting_heart_rate,smoking_status,daily_steps,stress_level,physical_activity_hours_per_week,sleep_hours,family_history_heart_disease,diet_quality_score,alcohol_units_per_week,heart_disease_risk_score,risk_category
0,1,62,25.0,142,93,247,72,Never,11565,3,5.6,8.2,No,7,0.7,28.1,Medium
1,2,54,29.7,158,101,254,74,Current,4036,8,0.5,6.7,No,5,4.5,63.0,High
2,3,46,36.2,170,113,276,80,Current,3043,9,0.4,4.0,No,1,20.8,73.1,High
3,4,48,30.4,153,98,230,73,Former,5604,5,0.6,8.0,No,4,8.5,39.5,Medium
4,5,46,25.3,139,87,206,69,Current,7464,1,2.0,6.1,No,5,3.6,29.3,Medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5495,5496,19,26.0,121,75,185,84,Never,6724,3,2.9,7.2,No,7,0.0,0.0,Low
5496,5497,18,30.9,128,82,235,75,Never,3661,4,0.0,5.5,No,1,9.6,16.8,Low
5497,5498,63,29.5,142,92,239,69,Never,6643,5,4.1,6.9,No,6,2.4,31.8,Medium
5498,5499,46,27.5,138,91,237,65,Never,3279,3,2.4,5.8,Yes,5,2.3,29.4,Medium


In [3]:
# checking information
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5500 entries, 0 to 5499
Data columns (total 17 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Patient_ID                        5500 non-null   int64  
 1   age                               5500 non-null   int64  
 2   bmi                               5500 non-null   float64
 3   systolic_bp                       5500 non-null   int64  
 4   diastolic_bp                      5500 non-null   int64  
 5   cholesterol_mg_dl                 5500 non-null   int64  
 6   resting_heart_rate                5500 non-null   int64  
 7   smoking_status                    5500 non-null   object 
 8   daily_steps                       5500 non-null   int64  
 9   stress_level                      5500 non-null   int64  
 10  physical_activity_hours_per_week  5500 non-null   float64
 11  sleep_hours                       5500 non-null   float64
 12  family

In [4]:
y = data['heart_disease_risk_score'].astype(int)


In [5]:
# drop unncessary columns
data.drop(['Patient_ID'], axis=1, inplace=True)

In [6]:
data.columns

Index(['age', 'bmi', 'systolic_bp', 'diastolic_bp', 'cholesterol_mg_dl',
       'resting_heart_rate', 'smoking_status', 'daily_steps', 'stress_level',
       'physical_activity_hours_per_week', 'sleep_hours',
       'family_history_heart_disease', 'diet_quality_score',
       'alcohol_units_per_week', 'heart_disease_risk_score', 'risk_category'],
      dtype='object')

In [13]:
from sklearn.preprocessing import OrdinalEncoder
# Controlling Data Leakage

''' 
1. Split the Dataset into X and y
2. Split the Dataset into Train and Test Sets
3. Use Encoding Techniques on Categorical Features
4. Use Scaling Techniques on Numerical Features
5. Use SMOTE
'''
# split the data into x and y
X = data.drop(['heart_disease_risk_score', 'risk_category'],axis=1)
y = data['risk_category']
# split the dataset into train and test 
X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                    random_state=1,
                                                    test_size= 0.3)

#segregate the categorical and numerical features
categorical_col = X_train.select_dtypes(include= 'object').columns
numerical_col = X_train.select_dtypes(exclude='object').columns

# apply label encoding

for i in categorical_col:
    encoder = OrdinalEncoder()
    X_train[i] = encoder.fit_transform(X_train[[i]])
    X_test[i] = encoder.transform(X_test[[i]])

# apply scaling techniques on numerical features
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
scaler = StandardScaler()
X_train[numerical_col] = scaler.fit_transform(X_train[numerical_col])
X_test[numerical_col] = scaler.transform(X_test[numerical_col])

In [14]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, BaggingClassifier
models = {
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'AdaBoost': AdaBoostClassifier(),
    'Bagging': BaggingClassifier()
}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"Model: {name}")
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Classification Report:\n", classification_report(y_test, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
    print("-" * 50)

Model: Random Forest
Accuracy: 0.8975757575757576
Classification Report:
               precision    recall  f1-score   support

        High       0.93      0.88      0.90       421
         Low       0.92      0.92      0.92       550
      Medium       0.87      0.89      0.88       679

    accuracy                           0.90      1650
   macro avg       0.90      0.90      0.90      1650
weighted avg       0.90      0.90      0.90      1650

Confusion Matrix:
 [[370   0  51]
 [  0 507  43]
 [ 30  45 604]]
--------------------------------------------------
Model: Gradient Boosting
Accuracy: 0.9157575757575758
Classification Report:
               precision    recall  f1-score   support

        High       0.95      0.88      0.91       421
         Low       0.94      0.93      0.94       550
      Medium       0.88      0.92      0.90       679

    accuracy                           0.92      1650
   macro avg       0.92      0.91      0.92      1650
weighted avg       0.92  